# Experiments ECAI24 on the approximation correctness

In [ ]:
# Basics
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from interval import *
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
tf.random.set_seed(1)
np.random.seed(1)

CYAN_COL = '\033[96m'
BLUE_COL = '\033[94m'
RED_COL = '\033[91m'
GREEN_COL = '\033[92m'
YELLOW_COL = '\033[93m'
RESET_COL = '\033[0m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'

In this experiment we test and confirm the correctness of our approximation. More specifically we consider this toy example to show that for a given $\delta$ discovered by our approximation and for which a MILP solver returns a non-robust result, with a confidence $\alpha$ could exist at most 1-R percentage of non-robust subINNs obtained from the original INN. 

<p align="center">
<img src="util_scripts/INN.png" alt="Drawing" align="center" style="width: 600px;"/>
</p>

In [2]:

def estimate_robustness(model, delta, cfx, concretizations, use_biases=True, robustness=True):

    """
    Utility method for the estimation of the CFX (not) Δ-robustness in the INN.

    Returns:
    --------
        rate: float
            estimation of the CFX (not) Δ-robustness computed with 'concretizations' models concretizations from the INN
    """
    np.random.seed(1)
    # Store initial weights
    old_weights = {}
    for l in range(1,len(model.layers)):
        old_weights[l] = model.layers[l].get_weights()

    for _ in range(concretizations):
        
        #perturbated_weights = {}
        input_features = np.array(cfx)

        for l in range(1,len(old_weights)+1):
            layer_weights = old_weights[l][0]
            if use_biases: layer_biases  = old_weights[l][1]
            
            weights_perturbation = np.random.uniform(-delta, delta, layer_weights.shape)
            if use_biases: biases_perturbation = np.random.uniform(-delta, delta, layer_biases.shape)
           
            
            layer_weights = [layer_weights+weights_perturbation]

            if use_biases: 
                layer_biases = [layer_biases+biases_perturbation]
                preactivated_res = np.dot(input_features, layer_weights) + layer_biases
            else:
                preactivated_res = np.dot(input_features, layer_weights)

            if l != len(old_weights):
                #relu
                activated_res = np.maximum(0.0, preactivated_res)
            else:
                #sigmoid
                activated_res = 1/(1 + np.exp(-preactivated_res))
            
            input_features = activated_res
            
        if input_features < 0.5:
            return 0  
    
    return 1

def compute_delta_max(model, cfx, delta_init, concretizations, use_biases=True, verbose=False):
  
    rate = estimate_robustness(model, delta_init, cfx, concretizations, use_biases)
    if rate != 1: return 0 # CFX not robust
        
    delta = delta_init
    while rate == 1: # for all the concretizations x results robust
        delta = 2*delta
        rate = estimate_robustness(model, delta, cfx, concretizations, use_biases)
        if verbose: 
            print(f'Testing δ={delta}')
            print(f'Rate is: {rate}')
    
    delta_max = delta/2
    
    while True:
        if abs(delta-delta_max) < delta_init:
            return delta_max

        if verbose: print(f"\nInterval to test is: [{delta_max}, {delta}]")
        
        delta_new = (delta_max+delta)/2
        rate = estimate_robustness(model, delta_new, cfx, concretizations, use_biases)
        if verbose: 
            print(f'Testing δ={delta_new}')
            print(f'Rate is: {rate}')
        
        if rate == 1:
            delta_max = delta_new
        else:
            delta = delta_new


def IBP(model, cfx, delta, use_biases=True):
    # Store initial weights
    old_weights = {}
    for l in range(1,len(model.layers)):
        old_weights[l] = model.layers[l].get_weights()
 
    # create input bounds from CFX for IBP
    input_bounds = []
    for elem in cfx:
        input_bounds.append(interval([elem, elem]))
    

    ### MAIN IBP LOOP
    for layer in range(1,len(old_weights)+1):
        layer_weights = old_weights[layer][0]
        layer_biases  = old_weights[layer][1]

        # every list in layer_weights is the list ok weights that exit from the i-th node of the previous layer. For instance if layer_weights is a list of 5 lists where each of the five list has 7 element, this means that the previous layer has 5 nodes, the next layer has 7 nodes, and the weights that start from the first node of the previous layer to the all the seven nodes of the next layer are stored in the first of the five lists.
        # we have to create for each weight an interval and replace the scalar weight with the interval.
        layer_weights_intervals = []
        for weights_starting_from_node in layer_weights:
            layer_intervals = []
            for weight in weights_starting_from_node:
                weight_interval = interval([weight-delta, weight+delta])
                layer_intervals.append(weight_interval)
            
            layer_weights_intervals.append(layer_intervals)

        if use_biases:
            layer_biases_intervals = []
            for i in range(len(layer_biases)):
                layer_biases_intervals.append(interval([layer_biases[i]-delta, layer_biases[i]+delta]))

      
        # now in layer_weights_intervals we have the same structure described above but with an interval instead of a scalar weight. And we proceed to generate the linear combination using interval bound multiplication 
        new_layer_bounds = {}
        index_input = -1
        for weight_intervals in layer_weights_intervals:
            index_input += 1
            index_new_input = 0
            for w_interval in weight_intervals:  
                if index_new_input in new_layer_bounds:
                    new_layer_bounds[index_new_input] += (w_interval*input_bounds[index_input])
                else:
                    new_layer_bounds[index_new_input] = (w_interval*input_bounds[index_input])

                index_new_input += 1

        if use_biases:
            for i in range(len(new_layer_bounds)):
                new_layer_bounds[i] += layer_biases_intervals[i]
       
        activated_bounds = []
        if layer != len(old_weights):
            #relu
            for node in new_layer_bounds:
                activated_bounds.append(interval(np.maximum(0.0, new_layer_bounds[node][0])))
        else:
            #sigmoid
            for node in new_layer_bounds:
                activated_bounds.append(1/(1 + np.exp(-new_layer_bounds[node][0][0])))

        input_bounds = activated_bounds
        #print(input_bounds)
        

    return input_bounds

  

def compute_formal_delta(model, cfx, delta_init, use_biases=True, verbose=False):
  
    lower = IBP(model, cfx, delta_init)[0]
    if lower < 0.5: return 0 # CFX not robust
        
    delta = delta_init
    while lower >= 0.5: # over-approx lower bound is >= 0.5, i.e., x results robust
        delta = 2*delta
        lower = IBP(model, cfx, delta, use_biases)[0]
        if verbose: 
            print(f'Testing δ={delta}')
            print(f'Lower is: {lower}')
    
    delta_max = delta/2
    
    while True:
        if abs(delta-delta_max) < delta_init:
            return delta_max

        if verbose: print(f"\nInterval to test is: [{delta_max}, {delta}]")
        
        delta_new = (delta_max+delta)/2
        lower = IBP(model, cfx, delta_new, use_biases)[0]
        if verbose: 
            print(f'Testing δ={delta_new}')
            print(f'Rate is: {lower}')
        
        if lower >= 0.5:
            delta_max = delta_new
        else:
            delta = delta_new



In [4]:
model = tf.keras.models.load_model('./models/toy_dnn_enum.h5', compile=False)
#model.summary()
delta_init = 0.0001

cfx = [[-2.57477835]]
print('cfx is:', cfx)
print('with pred:', model(np.array(cfx)).numpy().item())

delta_max_IBP = compute_formal_delta(model, cfx[0], delta_init, use_biases=False)
print('δ_max using IBP is', delta_max_IBP)

alpha = 0.9999
R = 0.9
print(f'Minimum concretization required with alpha={alpha} and R={R}:',int(np.emath.logn(R, (1-alpha))))
concretizations = 100000
delta_max_sampling = compute_delta_max(model, cfx, delta_init, concretizations, use_biases=False)
print('δ_max using Wilks is', delta_max_sampling)

cfx is: [[-2.57477835]]
with pred: 0.7032600045204163
δ_max using IBP is 0.10900000000000001
Minimum concretization required with alpha=0.9999 and R=0.9: 87
δ_max using Wilks is 0.11504999999999999


Let's write some useful methods...

In [5]:
class Node:

	def __init__(self, cfx, intervals, percentage=1, last_index_layer=0, last_node_interval=-1, robustness=True, parent=None):
		
		# Parameters
		self.cfx = cfx
		self.parent = parent
		self.intervals = intervals
		self.robustness = robustness
		self.percentage = percentage
		self.last_index_layer = last_index_layer
		self.last_node_interval = last_node_interval

	def IBP(self): 
		# create input bounds from CFX for IBP
		input_bounds = []
		for elem in self.cfx[0]:
			input_bounds.append(interval([elem, elem]))

		pre_activate_bounds = interval(np.maximum(0.0,(self.intervals[1][0]*input_bounds[0])[0]))*self.intervals[2][0] + interval(np.maximum(0.0,(self.intervals[1][1]*input_bounds[0])[0]))*self.intervals[2][1]
		lower_bound = 1/(1 + np.exp(-pre_activate_bounds[0][0]))
		upper_bound = 1/(1 + np.exp(-pre_activate_bounds[0][1]))
	
		return [lower_bound, upper_bound]
	
	def split_node(self):
		index_layer =  self.last_index_layer+1 if self.last_index_layer+1 <= 2 else 0
		node_interval = self.last_node_interval+1 if self.last_node_interval+1 <= 1 else 0
		
		interval_to_split = self.intervals[index_layer][node_interval]
		new_interval_left = interval([interval_to_split[0][0], interval_to_split.midpoint[0][0]])
		new_interval_right = interval([interval_to_split.midpoint[0][0], interval_to_split[0][1]])

		intervals_left = {}
		intervals_right = {}
		for key in self.intervals:
			intervals_left[key] = self.intervals[key].copy()
			intervals_right[key] = self.intervals[key].copy()
		
		new_list_left = intervals_left[index_layer]
		new_list_left[node_interval]= new_interval_left


		new_list_right = intervals_right[index_layer]
		new_list_right[node_interval]= new_interval_right
	
		intervals_left[index_layer] = new_list_left
		intervals_right[index_layer]= new_list_right

		return [Node(self.cfx, intervals_left, percentage=self.percentage/2, last_index_layer=self.last_index_layer, last_node_interval=self.last_node_interval, robustness=self.robustness, parent=self), Node(self.cfx, intervals_right, percentage=self.percentage/2, last_index_layer=self.last_index_layer, last_node_interval=self.last_node_interval, robustness=self.robustness, parent=self)] 

  
        

Let's create the INN

In [6]:
model = tf.keras.models.load_model('./models/toy_dnn_enum.h5', compile=False)
delta_max =  0.115
cfx = [[-2.57477835]]

# Store initial weights
old_weights = {}
for l in range(1,len(model.layers)):
    old_weights[l] = model.layers[l].get_weights()

layer_weights_intervals = {}
for layer in range(1,len(old_weights)+1):
    layer_weights = old_weights[layer][0]
   
    # every list in layer_weights is the list ok weights that exit from the i-th node of the previous layer. For instance if layer_weights is a list of 5 lists where each of the five list has 7 element, this means that the previous layer has 5 nodes, the next layer has 7 nodes, and the weights that start from the first node of the previous layer to the all the seven nodes of the next layer are stored in the first of the five lists.
    # we have to create for each weight an interval and replace the scalar weight with the interval.
    layer_intervals = []
    for weights_starting_from_node in layer_weights:
        for weight in weights_starting_from_node:
            weight_interval = interval([weight-delta_max, weight+delta_max])
            layer_intervals.append(weight_interval)
        
        layer_weights_intervals[layer] = layer_intervals

In [7]:
print(layer_weights_intervals)

{1: [interval([-0.480079402923584, -0.250079402923584]), interval([-0.9888380074501038, -0.7588380074501038])], 2: [interval([-1.13864981174469, -0.90864981174469]), interval([0.6961773729324341, 0.9261773729324341])]}


Starting with the enumeration...

In [8]:
starting_INN = Node(cfx, layer_weights_intervals)
frontier = [starting_INN]
next_frontier = []
max_depth = 40
percentage_not_robust = 0
percentage_robust = 0
R = 0.9

for depth in range(1,max_depth+1):
    for INN in frontier:
        lower, upper = INN.IBP()
      
        if upper < 0.5:
            # not robust
            percentage_not_robust += INN.percentage
           
        elif lower >= 0.5:
            #robust
            percentage_robust += INN.percentage
            
        else:
            INN_1, INN_2 = INN.split_node()   
            next_frontier.append(INN_1)
            next_frontier.append(INN_2)

    # The tree has been completely explored
    if frontier == []: break

    # Update the frontier with the new unexplored nodes
    frontier.clear()
    frontier = next_frontier.copy()
    next_frontier.clear()

    print("__________________________________________________________________\n")
    print( f"{YELLOW_COL}[Depth reached]:{RESET_COL} {depth}/{max_depth}")
    print( f"\t{BLUE_COL}[Monitor]{RESET_COL} % robust: {percentage_robust*100}")
    print( f"\t{BLUE_COL}[Monitor]{RESET_COL} % non-robust: {percentage_not_robust*100}")
    print( f"\t{BLUE_COL}[Monitor]{RESET_COL} Nodes to explore: {len(frontier)}")
    max_non_robust = (1/(2**(depth))) * len(frontier)
    print( f"\t{BLUE_COL}[Monitor]{RESET_COL} Maximum % non-robust achievable is: {max_non_robust}")
    print( f"\t{BLUE_COL}[Monitor]{RESET_COL} The % of INNs non-robust could be > 1-R?: {max_non_robust > 1-R}")

    if max_non_robust <= 1-R:
        break

print( f"\t{GREEN_COL} % robust: {percentage_robust}")
print( f"\t{GREEN_COL} % non-robust: {percentage_not_robust}")

__________________________________________________________________

[Depth reached]: 1/40
	[Monitor] % robust: 0
	[Monitor] % non-robust: 0
	[Monitor] Nodes to explore: 2
	[Monitor] Maximum % non-robust achievable is: 1.0
	[Monitor] The % of INNs non-robust could be > 1-R?: True
__________________________________________________________________

[Depth reached]: 2/40
	[Monitor] % robust: 50.0
	[Monitor] % non-robust: 0
	[Monitor] Nodes to explore: 2
	[Monitor] Maximum % non-robust achievable is: 0.5
	[Monitor] The % of INNs non-robust could be > 1-R?: True
__________________________________________________________________

[Depth reached]: 3/40
	[Monitor] % robust: 75.0
	[Monitor] % non-robust: 0
	[Monitor] Nodes to explore: 2
	[Monitor] Maximum % non-robust achievable is: 0.25
	[Monitor] The % of INNs non-robust could be > 1-R?: True
__________________________________________________________________

[Depth reached]: 4/40
	[Monitor] % robust: 87.5
	[Monitor] % non-robust: 0
	[Monitor]